# Tarefa 4: interface conversacional: conversar com os LLMs Llama 3 e Titan Premier

Neste caderno, você criará um chatbot usando os modelos de base (FMs) llama3-8b-instruct e titan-text-premier no Amazon Bedrock.

Interfaces conversacionais, como chatbots e assistentes virtuais, podem aprimorar a experiência do usuário para seus clientes. Os chatbots usam algoritmos de processamento de linguagem natural (PLN) e machine learning para entender e responder às consultas dos usuários. Você pode usar chatbots em diversas aplicações, como atendimento ao cliente, vendas e comércio eletrônico, para oferecer respostas rápidas e eficientes aos usuários. Os usuários podem acessá-los por vários canais, como sites, plataformas de redes sociais e aplicativos de mensagens.

- **Chatbot (básico)**, chatbot zero-shot com modelo FM
- **Chatbot com uso de prompt**, template(LangChain): chatbot com algum contexto fornecido no modelo de prompt
- **Chatbot com persona**: chatbot com funções definidas, por exemplo, coach de carreira e interações humanas
- **Chatbot com consciência contextual**, inclusão de contexto por meio de um arquivo externo gerando incorporações.

## Framework LangChain para criação de chatbot com o Amazon Bedrock

Em interfaces conversacionais, como chatbots, lembrar as interações anteriores é muito importante, tanto no curto quanto no longo prazo.

O framework LangChain fornece componentes de memória em duas formas. Primeiro, o LangChain oferece recursos auxiliares para gerenciar e manipular mensagens de chat anteriores. Eles são projetados para serem modulares. Em segundo lugar, o LangChain fornece maneiras fáceis de incorporar esses utilitários em cadeias, permitindo que você defina e interaja facilmente com diferentes tipos de abstrações, o que facilita a criação de chatbots avançados.

## Criar chatbot com contexto: elementos principais

O primeiro processo na criação de um chatbot com consciência contextual é gerar incorporações para o contexto. Normalmente, ocorre um processo de ingestão no seu modelo para gerar as incorporações, que depois são inseridas no armazenamento de vetores. Neste caderno, você usa o modelo do Titan Embeddings para isso. O segundo processo é a orquestração, interação, invocação e o retorno dos resultados das solicitações dos usuários. Isso envolve orquestrar a solicitação do usuário, interagir com os modelos/componentes necessários para coletar informações, invocar o chatbot para formular uma resposta e, em seguida, retornar a resposta do chatbot ao usuário.

## Tarefa 4.1: Configurar o ambiente

Nesta tarefa, você configurará o ambiente.

In [15]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


In [16]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Formate as instruções onde os papéis na conversa devem se alternar system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

## Tarefa 4.2: usar o histórico do chat do LangChain para iniciar a conversa

Nesta tarefa, você ativará o chatbot para que guarde o contexto das conversas entre as várias interações com os usuários. Ter uma memória conversacional é imprescindível para que os chatbots mantenham diálogos significativos e coerentes ao longo do tempo.

Você implementa recursos de memória conversacional com base na classe InMemoryChatMessageHistory do LangChain. Esse objeto armazena as conversas entre o usuário e o chatbot, e o histórico fica disponível para que o agente do chatbot possa aproveitar o contexto de uma conversa anterior.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** as saídas do modelo não são determinísticas.

In [17]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock
from IPython.display import display, Markdown

chat_model=ChatBedrock(
    model_id="us.amazon.nova-premier-v1:0" , 
    client=bedrock_client)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Responda às seguintes perguntas da melhor forma possível."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
query="como vai você?"
response=wrapped_chain.invoke({"input": query})
# Printing history to see the history being built out. 
for msg in history.messages:
    display(Markdown(f"**{msg.type.upper()}:** {msg.content}"))
    print("-" * 80)

# For the rest of the conversation, the output will only include response

**HUMAN:** como vai você?

--------------------------------------------------------------------------------


**AI:** Eu não tenho sentimentos ou emoções, mas agradeço por perguntar! Estou aqui e pronto para ajudar com qualquer dúvida ou informação que você precise. Como posso ser útil para você hoje? Se houver algo específico que você queira saber ou discutir, é só me avisar!

--------------------------------------------------------------------------------


### Novas perguntas

O modelo respondeu com uma mensagem inicial. Agora, você faz algumas perguntas.

In [18]:
#new questions
instructions = [{"role": "user", "content": "Me dê algumas dicas de como começar uma nova horta."}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
display(Markdown(response))


Claro! Começar uma horta pode ser uma experiência gratificante. Aqui estão algumas dicas para ajudar você a iniciar:

1. **Escolha o Local**:
   - **Sol**: A maioria das plantas precisa de pelo menos 6 horas de luz solar direta por dia.
   - **Acesso à Água**: Proximidade a uma fonte de água facilita a irrigação.

2. **Planejamento**:
   - **Tipos de Plantas**: Decida quais vegetais, ervas ou frutas você quer cultivar. Considere o clima da sua região.
   - **Espaço**: Determine quanto espaço você tem disponível e planeje a disposição das plantas.

3. **Preparação do Solo**:
   - **Teste o Solo**: Um teste de solo pode ajudar a determinar o pH e quais nutrientes são necessários.
   - **Melhorias**: Adicione composto ou esterco para enriquecer o solo.

4. **Semeadura ou Transplante**:
   - **Sementes ou Mudas**: Decida se vai começar a partir de sementes ou comprar mudas.
   - **Profundidade e Espaçamento**: Siga as instruções de plantio específicas para cada tipo de planta.

5. **Irrigação**:
   - **Rega Regular**: Mantenha o solo úmido, mas não encharcado. A frequência da rega depende do clima e do tipo de solo.
   - **Sistema de Irrigação**: Considere sistemas de gotejamento ou aspersão para uma rega mais eficiente.

6. **Manutenção**:
   - **Capina**: Remova ervas daninhas regularmente.
   - **Adubação**: Aplique fertilizantes conforme necessário, baseando-se nas necessidades das plantas.

7. **Controle de Pragas e Doenças**:
   - **Monitoramento**: Fique atento a sinais de pragas e doenças.
   - **Métodos Naturais**: Considere o uso de inseticidas naturais e práticas como rotação de culturas.

8. **Colheita**:
   - **Tempo Certo**: Colha os vegetais no momento certo para garantir o melhor sabor e qualidade.

Começar uma horta requer um pouco de planejamento e esforço, mas os resultados podem ser muito gratificantes. Boa sorte e bom cultivo!

### Aprimorar as perguntas

Agora, faça uma pergunta sem mencionar a palavra jardim para ver se a modelo consegue entender a conversa anterior.

In [19]:
# build on the questions
instructions = [{"role": "user", "content": "insetos"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
display(Markdown(response))


Os insetos são um grupo diverso de animais invertebrados que pertencem à classe Insecta, dentro do filo Arthropoda. Eles são os animais mais numerosos do planeta, com mais de um milhão de espécies descritas, representando cerca de dois terços de todos os organismos conhecidos.

### Características Principais dos Insetos:
1. **Corpo Segmentado**: Dividido em três partes principais: cabeça, tórax e abdômen.
2. **Três Pares de Patas**: Localizadas no tórax, permitindo mobilidade.
3. **Um Par de Antenas**: Sensoriais, ajudando na detecção de odores e vibrações.
4. **Exoesqueleto**: Uma estrutura externa rígida que oferece proteção e suporte.
5. **Asas**: Muitos insetos têm asas, geralmente dois pares, permitindo o voo.

### Diversidade e Habitat:
Insetos habitam praticamente todos os ambientes da Terra, desde florestas tropicais até desertos, e de águas doces a ambientes urbanos. Essa adaptabilidade é uma das razões para sua grande diversidade.

### Importância Ecológica:
- **Polinização**: Muitos insetos, como abelhas e borboletas, são polinizadores essenciais para a reprodução de plantas.
- **Decomposição**: Insetos como besouros e formigas ajudam a decompor matéria orgânica, reciclando nutrientes.
- **Controle Biológico**: Alguns insetos predam outras pragas, ajudando a manter o equilíbrio ecológico.

### Importância Econômica:
- **Agricultura**: Além da polinização, insetos podem ser benéficos (por exemplo, joaninhas que comem pulgões) ou prejudiciais (por exemplo, gafanhotos que destroem colheitas).
- **Medicina**: Alguns insetos são usados na produção de medicamentos e pesquisas.

### Exemplos de Insetos:
- **Formigas**: Sociais e altamente organizadas.
- **Borboletas e Mariposas**: Importantes polinizadores.
- **Moscas e Mosquitos**: Alguns podem ser vetores de doenças.
- **Besouros**: O grupo mais diverso de insetos.

Os insetos desempenham papéis cruciais em ecossistemas e na vida humana, tornando-os um grupo de estudo fascinante e vital para a compreensão da biologia e ecologia.

### Encerrar a conversa

In [20]:
# finishing the conversation
instructions = [{"role": "user", "content": "É só isso, obrigado!"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
display(Markdown(response)) 

Você é sempre bem-vindo! Se tiver mais perguntas ou precisar de ajuda no futuro, não hesite em retornar. Estou aqui para ajudar com qualquer informação ou dúvida que você possa ter. Tenha um ótimo dia e sucesso com a sua horta!

## Tarefa 4.3: chatbot usando modelo de prompt (LangChain)

Nesta tarefa, você usará o PromptTemplate padrão que é responsável pela elaboração dessa entrada. O LangChain oferece várias classes e funções para facilitar a elaboração e trabalho com prompts.

In [21]:
#  prompt for a conversational agent
def format_prompt(actor:str, input:str):
    formatted_prompt: List[str] = []
    if actor == "system":
        prompt_template="""<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    elif actor == "user":
        prompt_template="""<|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    else:
        raise ValueError(f"Invalid role: {actor}. Role must be either 'user' or 'system'.")   
    prompt = PromptTemplate.from_template(prompt_template)     
    formatted_prompt.extend(prompt.format(actor=actor,input=input))
    formatted_prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(formatted_prompt)

In [27]:
# chat user experience
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            with self.out:
                print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        response = self.qa.invoke({"input": prompt})
                        result=response['answer']
                    else:
                        instructions = [{"role": "user", "content": prompt}]
                        #result = self.qa.invoke({'input': format_prompt("user",prompt)}) #, 'history':chat_history})
                        result = self.qa.invoke({"input": format_instructions(instructions)})
                except:
                    result = "No answer"
                thinking.value=""
                display(Markdown(result))
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Em seguida, inicie uma conversa.

In [28]:
# start chat
history = InMemoryChatMessageHistory() #reset chat history
chat = ChatUX(wrapped_chain)
chat.start_chat()

Starting chat bot


Output()

In [29]:
print(history)

Human: <|start_header_id|>user<|end_header_id|>
como se emaranha duas particula atomicas <|eot_id|><|start_header_id|>assistant<|end_header_id|>

AI: Emaranhar duas partículas atômicas, como elétrons, átomos, ou fótons, é um conceito fundamental na mecânica quântica conhecido como "emaranhamento quântico". Esse fenômeno ocorre quando duas partículas se tornam interligadas de tal maneira que o estado de uma partícula não pode ser descrito independentemente do estado da outra, mesmo que as duas estejam separadas por grandes distâncias.

Aqui está um resumo simplificado de como o emaranhamento pode ser alcançado:

### 1. Criação de um Estado Emaranhado
Para criar um par de partículas emaranhadas, geralmente se começa com uma interação física entre elas. Isso pode ser feito de várias maneiras, dependendo do tipo de partículas envolvidas.

#### a) Emaranhamento de Fótons:
Uma maneira comum de emaranhar fótons é através do uso de um cristal não-linear em um processo chamado conversão paramét

## Tarefa 4.4: Chatbot com persona

Nesta tarefa, o assistente de Inteligência Artificial (IA) desempenha a função de coach de carreira. Você pode usar uma mensagem do sistema para informar ao chatbot qual persona (ou função) ele vai desempenhar. Continue aproveitando a classe InMemoryChatMessageHistory para manter o contexto da conversa.

In [33]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " Irá atuar como um consultor de carreira. O seu objetivo é dar conselhos de carreira aos utilizadores. Para questões que não estejam relacionadas com a carreira, não dê conselhos. Diga: Não sei"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory() # reset history

chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="career_chat_history",
)

response=wrapped_chain.invoke({"input": "Quais são as opções de carreira em IA?"})
#print(response)
display(Markdown(response))

Existem várias áreas promissoras na Inteligência Artificial (IA) que você pode considerar, como cientista de dados, engenheiro de machine learning, especialista em visão computacional, e pesquisador de IA. Cada uma dessas áreas requer habilidades específicas e oferece oportunidades únicas para contribuir com o avanço da tecnologia. É importante avaliar suas habilidades em matemática, estatística e programação, além de considerar seu interesse em pesquisa e desenvolvimento ao escolher um caminho na IA.

In [35]:
response=wrapped_chain.invoke({"input": "Como consertar meu carro?"})
display(Markdown(response))

Desculpe, mas não posso responder a questões que não estejam relacionadas com desenvolvimento de carreira. Sugiro procurar um mecânico de confiança ou consultar o manual do proprietário do seu veículo para dicas de manutenção básica. Se estiver procurando aprimorar habilidades profissionais relacionadas à mecânica automotiva, posso ajudar com conselhos sobre cursos e certificações na área.

In [36]:
print(history)

Human: Quais são as opções de carreira em IA?
AI: Existem várias áreas promissoras na Inteligência Artificial (IA) que você pode considerar, como cientista de dados, engenheiro de machine learning, especialista em visão computacional, e pesquisador de IA. Cada uma dessas áreas requer habilidades específicas e oferece oportunidades únicas para contribuir com o avanço da tecnologia. É importante avaliar suas habilidades em matemática, estatística e programação, além de considerar seu interesse em pesquisa e desenvolvimento ao escolher um caminho na IA.
Human: Como consertar meu carro?
AI: Não sei responder a essa pergunta, pois não está relacionada a consultoria de carreira. Se você tiver alguma dúvida sobre como desenvolver habilidades profissionais ou escolher um caminho de carreira, ficarei feliz em ajudar. Por exemplo, se estiver interessado em mecânica automotiva como profissão, posso fornecer informações sobre como iniciar nessa área.
Human: Como consertar meu carro?
AI: Desculpe, 

Agora, faça uma pergunta que não esteja dentro da especialidade dessa persona. O modelo não deve responder a essa pergunta e deve dar uma razão para isso.

## Tarefa 4.5 Chatbot com contexto

Nesta tarefa, você pedirá que o chatbot responda perguntas com base no contexto que foi passado para ele. Você usa um arquivo CSV e o modelo do Titan Embeddings para criar um vetor representando esse contexto. Esse vetor é armazenado no Facebook AI Similarity Search (FAISS). Quando uma pergunta for feita ao chatbot, você transmitirá esse vetor de volta para o chatbot e fará com que ele recupere a resposta usando o vetor.

### Modelo de incorporações do Titan

As incorporações representam palavras, frases ou qualquer outro item distinto como vetores em um espaço de vetores contínuo. Assim, os modelos de machine learning podem fazer operações matemáticas nessas representações e capturar as relações semânticas entre elas.

Você usa incorporações para a geração aumentada de recuperação (RAG) [recurso de pesquisa de documentos](https://labelbox.com/blog/how-vector-similarity-search-works/).

In [37]:
# model configuration
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

### FAISS como VectorStore

Para usar incorporações na pesquisa, você precisa de um armazenamento que faça pesquisas por similaridade de vetores de forma eficaz. Neste caderno, você usa o FAISS, que é um armazenamento em memória. Para armazenar vetores permanentemente, você pode usar as Bases de Conhecimento para Amazon Bedrock, pgVector, Pinecone, Weaviate ou Chroma.

As APIs VectorStore do LangChain estão disponíveis [aqui](https://python.langchain.com/v0.2/docs/integrations/vectorstores/).

In [42]:
# vector store
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

loader = CSVLoader("./rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

documents:loaded:size=153
Documents:after split and chunking size=154
vectorstore_faiss_aws:created=<langchain_community.vectorstores.faiss.FAISS object at 0x00000197978A34D0>::


### Fazer um teste rápido com pouco código 

Você pode usar uma classe wrapper fornecida pelo LangChain para consultar o armazenamento do banco de dados de vetores e retornar os documentos relevantes. Isso processa uma cadeia de QA com todos os valores padrão.

In [44]:
chat_llm=ChatBedrock(
    model_id="us.amazon.nova-premier-v1:0" , 
    client=bedrock_client)
# wrapper store faiss
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
display(Markdown(wrapper_store_faiss.query("O SageMaker suporta o R", llm=chat_llm)))

Sim, o Amazon SageMaker suporta a linguagem de programação R. Ele oferece kernels de Jupyter Notebook para R, permitindo que os usuários escrevam e executem scripts em R para tarefas de machine learning. Você pode usar o R no SageMaker para preparar dados, treinar modelos, e fazer deploy, tudo dentro de um ambiente totalmente gerenciado. Além disso, o SageMaker Studio, que é a interface web integrada para desenvolvimento de ML, também suporta notebooks em R, facilitando ainda mais o trabalho com essa linguagem na plataforma.

### Aplicação de chatbot

Para o chatbot, você precisa de gerenciamento de contexto, histórico, armazenamentos de vetores e muitos outros componentes. Comece criando uma cadeia de Geração aumentada de recuperação (RAG) com suporte a contexto.

Ela usa as funções **create_stuff_documents_chain** e **create_retrieval_chain**

### Parâmetros e funções usados para a RAG

- **Recuperador:** você usou `VectorStoreRetriever`, que é apoiado por um `VectorStore`. Para recuperar texto, há dois tipos de pesquisa para escolher: `"similarity"` ou `"mmr"`. `search_type="similarity"` usa a pesquisa por similaridade no objeto recuperador, onde seleciona vetores de fragmentos de texto que são mais semelhantes ao vetor da pergunta.

- **create_stuff_documents_chain** especifica como o contexto recuperado é alimentado em um prompt e no LLM. Os documentos recuperados são "preenchidos" como contexto sem qualquer resumo ou outro processamento no prompt.

- **create_retrieval_chain** adiciona a etapa de recuperação, propaga o contexto recuperado pela cadeia e o fornece junto com a resposta final. 

Se a pergunta feita estiver fora do escopo do contexto, o modelo responderá que não sabe a resposta.

In [45]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Você é um assistente para as tarefas de resposta a perguntas."
    "Utilize os seguintes excertos de contexto recuperados para responder à questão."
    "Se não souber a resposta, diga que não sabe."
    "Utilize no máximo três frases e mantenha a resposta concisa."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever=vectorstore_faiss_aws.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "O que é o Sagemaker?"})
print(response) # shows the document chunks consulted to come up with the answer

{'input': 'O que é o Sagemaker?', 'context': [Document(id='5cd86b93-bf5d-43aa-8986-ff1b203230d1', metadata={'source': './rag_data/Amazon_SageMaker_FAQs.csv', 'row': 12}, page_content='ï»¿What is Amazon SageMaker?: What is Amazon SageMaker Studio?\nAmazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.: Amazon SageMaker Studio provides a single, web-based visual interface where you can perform all ML development steps. SageMaker Studio gives you complete access, control, and visibility into each step required to prepare data and build, train, and deploy models. You can quickly upload data, create new notebooks, train and tune models, move back and forth between steps to adjust experiments, compare results, and deploy models to production all in one place, making you much more productive. All ML development activities including notebooks, experiment man

Em seguida, inicie uma conversa

In [46]:
chat = ChatUX(rag_chain, retrievalChain=True)
chat.start_chat()  # Only answers will be shown here, and not the citations

Starting chat bot


Output()


Você usou o LLM Titan para criar uma interface conversacional com os seguintes padrões:

- Chatbot (básico: sem contexto)
- Chatbot usando modelo de prompt (LangChain)
- Chatbot com personas
- Chatbot com contexto

### Experimente você mesmo

- Altere os prompts para seu caso de uso específico e avalie o resultado de diferentes modelos.
- Teste o comprimento do token para entender a latência e a responsividade do serviço.
- Aplique diferentes princípios de engenharia de prompts para gerar resultados melhores.

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de caderno e continue com a **Tarefa 5**.